PARCIAL I - Maria Fernanda Mora Varón

Nota : El factor de expancion es F_EXP

In [1]:
#Importanto las librerias necesarias para el analisi de la GEIH
import pandas as pd
import numpy as np

In [2]:
#Importar datos 
df_emicron=pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/EMICRON/emicron_identificacion.csv", encoding="latin", sep=",")

In [3]:
df_emicron.head()

,DIRECTORIO,SECUENCIA_P,SECUENCIA_ENCUESTA,COD_DEPTO,AREA,CLASE_TE,P35,P241,MES_REF,P3031,P3032_1,P3032_2,P3032_3,P3033,P3034,P3035,P3000,GRUPOS4,GRUPOS12,F_EXP
0,7627444,1,1,44,NaN,2,2,33,ENERO,2,NaN,NaN,NaN,2,240,2,2,2,3,60.050515
1,7627446,1,1,44,NaN,2,1,31,ENERO,2,NaN,NaN,NaN,2,166,2,2,1,1,86.341075
2,7627449,1,1,68,NaN,1,2,42,ENERO,2,NaN,NaN,NaN,2,60,2,2,3,5,139.884518
3,7627453,1,2,68,NaN,1,2,41,ENERO,2,NaN,NaN,NaN,2,60,2,2,4,7,168.635440
4,7627456,1,3,68,NaN,1,1,18,ENERO,2,NaN,NaN,NaN,2,9,2,2,4,6,64.659273


In [4]:
total = df_emicron["F_EXP"].sum()
print("Población total expandida:", int(total))


Población total expandida: 5297251


In [11]:
#Primero tenemos que determinar que variable vamso utilizar
df_emicron.columns

Index(['DIRECTORIO', 'SECUENCIA_P', 'SECUENCIA_ENCUESTA', 'COD_DEPTO', 'AREA',
       'CLASE_TE', 'P35', 'P241', 'MES_REF', 'P3031', 'P3032_1', 'P3032_2',
       'P3032_3', 'P3033', 'P3034', 'P3035', 'P3000', 'GRUPOS4', 'GRUPOS12',
       'F_EXP'],
      dtype='object')

PREGUNTAS : 
1. ¿Cuantos micronegocios existen en el quindio cuyos propietarios son hombres del quindio son entre 18 y 30 años ?

In [10]:
# Filtrar Quindío (COD_DEPTO=63), hombres (P35=1), edad entre 18 y 30
filtro = df_emicron[
    (df_emicron["COD_DEPTO"] == 63) &
    (df_emicron["P35"] == 1) &
    (df_emicron["P241"].between(18, 30))
]

# Calcular micronegocios expandidos
total_quindio_hombres_18_30 = filtro["F_EXP"].sum()

print("Los micronegocios que hay en el Quindío con propietarios hombres entre los 18 y 30 años:",
      round(total_quindio_hombres_18_30))


Los micronegocios que hay en el Quindío con propietarios hombres entre los 18 y 30 años: 1929


2. ¿cuantos micronegocios de la industria manufacturera existen en el departanmento de risaralda, cuyos propietarias son mujeres entre los 20 y 30 años?


In [14]:
# Micronegocios de industria manufacturera en Risaralda
# con propietarias mujeres entre 20 y 30 años
#Ahora agregamos la varibale de interes que es GRUPOS4 que nos permite saber de que sector es la microempresa
#GRUPO4=2 Industria manufacturera 

#Agregamos estos codigos pero ojo no cambiamos el P35 a 2 para referirnos a mujeres y el DPTO a 66 para referirnos a Risaralda 
micronegocio_manu_fr = df_emicron[(df_emicron["COD_DEPTO"] == 66) & (df_emicron["P35"] == 2) & (df_emicron["P241"].between(20, 30)) & 
                                  (df_emicron['GRUPOS4'] == 2)]


# Calcular micronegocios con F_EXP
total_microMFR = micronegocio_manu_fr["F_EXP"].sum()

print(f"Los micronegocios de la industria manufacturera en Risaralda que son mujeres entre los 20 y 30 años :", int(total_microMFR))

Los micronegocios de la industria manufacturera en Risaralda que son mujeres entre los 20 y 30 años : 471


3. ¿Entre las cuidades de manizales, pereira y armenia. Cual de las 3 tieene mas micronegocios del sector comercio, cuyas propetarias son mujeres mayores a 40 años?

In [16]:
#Ahora es lo mismp pero cambiamos 
#GRUPO4 = 3 para el comercio
# usamos AREA para las ciudades de interes
#Pereira es el primero 
micronegocio_COM_FP40 = df_emicron[(df_emicron["AREA"] == 66) & (df_emicron["P35"] == 2) & (df_emicron["P241"] > 40) & 
                                  (df_emicron['GRUPOS4'] == 3)]

total_microCOFP40 = micronegocio_COM_FP40['F_EXP'].sum()

print(f'Los micronegocios de Pereira son:', int(total_microCOFP40))

#Ahora con Armenia
micronegocio_com_FA40 = df_emicron[(df_emicron["AREA"] == 63) & (df_emicron["P35"] == 2) & (df_emicron["P241"] > 40) & 
                                  (df_emicron['GRUPOS4'] == 3)]

total_microCOFA40 = micronegocio_com_FA40['F_EXP'].sum()

print(f'Los micronegocios de Armenia son:', int(total_microCOFA40))

#Ahora con manizales 
micronegocio_com_FM40 = df_emicron[(df_emicron["AREA"] == 17) & (df_emicron["P35"] == 2) & (df_emicron["P241"] > 40) & 
                                  (df_emicron['GRUPOS4'] == 3)]

total_microCOFM40 = micronegocio_com_FM40['F_EXP'].sum()

print(f'Los micronegocios de Manizales son:',int(total_microCOFM40))
print()

Los micronegocios de Pereira son: 3827
Los micronegocios de Armenia son: 2557
Los micronegocios de Manizales son: 2636



4. ¿En la cuidad de armenia, cuantos micronegocios hay cuyos propetrios son hombres que no tiene  nombre comercial?

In [18]:
#Ya por ultimo utilicemos la nueva variable  P3035 esta nos determina si tiene nombre comercial o no 
# 1 para si y 2 para no

micronegocioNN = df_emicron[(df_emicron["AREA"] == 63) & (df_emicron["P35"] == 1) & (df_emicron["P3035"] == 2)]
total_microNN = micronegocioNN['F_EXP'].sum()

print(f'EN Armenia hay un total de', int(total_microNN), f'que son micronegocios cuyos propietarios son hombres y que no tiene nombre comercial')


EN Armenia hay un total de 11603 que son micronegocios cuyos propietarios son hombres y que no tiene nombre comercial
